<a href="https://colab.research.google.com/github/tanatet8/Colab_Script/blob/main/notion_expense_sync_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 🔢 Block 1: ติดตั้งไลบรารี
!pip install gspread oauth2client notion-client python-dotenv

In [2]:
# 🔢 Block 2: Mount Google Drive และโหลด .env
from google.colab import drive
drive.mount('/content/drive')

from dotenv import load_dotenv
import os

# ✅ แก้ตรงนี้ให้ถูก
env_path = "/content/drive/MyDrive/GAS_Automation_Hub/Automation_Keys/Notion_Expense/.env"  # ← ใส่จุดนำหน้า
load_dotenv(dotenv_path=env_path, override=True)

# โหลดตัวแปรจาก .env
SHEET_ID = os.getenv("SHEET_ID")
NOTION_TOKEN = os.getenv("NOTION_TOKEN")
NOTION_DB_ID = os.getenv("NOTION_DB_ID")

# 🔍 ทดสอบ
print("✅ SHEET_ID =", SHEET_ID)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ SHEET_ID = 1j9GE9KsQnZdUVohsDJ9e97FDqQC-g1-Cv81L1olnHu4


In [3]:
!ls "/content/drive/MyDrive/GAS_Automation_Hub/Automation_Keys/Notion_Expense"

 service_account.json  'Untitled document.gdoc'


In [4]:
# 🔢 Block 3: เชื่อม Google Sheets

import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# ❗ แก้ path ให้ตรงกับไฟล์จริง
service_account_path = "/content/drive/MyDrive/GAS_Automation_Hub/Automation_Keys/Notion_Expense/service_account.json"

# เชื่อม Google Sheets
creds = ServiceAccountCredentials.from_json_keyfile_name(service_account_path, scope)
client = gspread.authorize(creds)
sheet = client.open_by_key(SHEET_ID).sheet1

print("✅ เชื่อม Google Sheet สำเร็จ:", sheet.title)


✅ เชื่อม Google Sheet สำเร็จ: Sheet1


In [5]:
# 🔢 Block 4: ตั้งค่า keyword + Mapping database สำหรับ Notion
dynamic_keywords = {
    "อาหาร": ["ข้าว", "แกง", "ก๋วยเตี๋ยว", "กาแฟ", "ชา", "ไข่", "นม", "ขนม", "โจ๊ก"],
    "ของใช้": ["สบู่", "แชมพู", "ผ้า", "ทิชชู่", "ยาสีฟัน", "ครีม"],
    "รถ": ["น้ำมัน", "รถเมล์", "จอดรถ", "เติมลม", "ค่าทางด่วน"],
    "สุขภาพ": ["ยา", "คลินิก", "หมอ", "ฟิตเนส"],
    "บริจาค": ["ทำบุญ", "บริจาค", "ถวาย", "ตักบาตร"],
    "เงินสด": ["ถอน", "เบิก", "รับเงิน", "โอน"],
}

category_to_db_id = {
    "อาหาร": os.getenv("NOTION_DB_ID_FOOD"),
    "ของใช้": os.getenv("NOTION_DB_ID_SHOPPING"),
    "รถ": os.getenv("NOTION_DB_ID_TRANSPORT"),
    "สุขภาพ": os.getenv("NOTION_DB_ID_HEALTH"),
    "บริจาค": os.getenv("NOTION_DB_ID_DONATION"),
    "เงินสด": os.getenv("NOTION_DB_ID_CASH"),
}

In [6]:
# 🔢 Block 5: ฟังก์ชันพื้นฐาน → แยกข้อความ + เดาหมวดหมู่
def parse_message(text):
    try:
        parts = text.strip().split()
        if len(parts) == 2:
            return parts[0], float(parts[1])
        elif len(parts) == 1:
            return parts[0], 0.0
        else:
            return None, None
    except:
        return None, None

def find_category(text):
    for category, keywords in dynamic_keywords.items():
        if any(word in text for word in keywords):
            return category
    return None


In [7]:
# 🔢 Block 6: เชื่อม Notion และ push ธรรมดา
from notion_client import Client
notion = Client(auth=NOTION_TOKEN)

def push_to_notion(category, amount):
    notion.pages.create(
        parent={"database_id": NOTION_DB_ID},
        properties={
            "Categories": {"select": {"name": category}},
            "Amount": {"number": amount}
        }
    )

In [8]:
# 🔢 Block 7: sync Google Sheet → Notion (กรณีใช้หมวดชัดเจน)
def sync_sheet_to_notion():
    data = sheet.get_all_records()
    for idx, row in enumerate(data):
        if row['Synced?'].strip().upper() != "NO":
            continue

        category, amount = parse_message(row['Message'])
        if not category or amount is None:
            print(f"❌ รูปแบบผิด row {idx+2}: {row['Message']}")
            continue

        try:
            push_to_notion(category, amount)
            sheet.update_cell(idx+2, 4, "YES")
            print(f"✅ Synced row {idx+2}: {category} {amount}")
        except Exception as e:
            print(f"🚨 Error at row {idx+2}: {e}")

In [9]:
# 🔢 Block 8: Push แบบเดาหมวดหมู่จากข้อความ
def push_to_notion_dynamic(message):
    category, amount = parse_message(message)
    if not category or amount is None:
        print(f"❌ ข้อความไม่ถูกต้อง: '{message}'")
        return

    guessed_category = find_category(category)
    if not guessed_category:
        print(f"⚠️ ไม่พบหมวดสำหรับ '{category}'")
        return

    db_id = category_to_db_id.get(guessed_category)
    if not db_id:
        print(f"❌ ไม่มี DB สำหรับ '{guessed_category}'")
        return

    notion.pages.create(
        parent={"database_id": db_id},
        properties={
            "Name": {"title": [{"text": {"content": message}}]},
            "Amount": {"number": amount}
        }
    )
    print(f"✅ ส่งข้อมูลสำเร็จ: '{message}' → '{guessed_category}'")

In [10]:
# 🔢 Block 9: เก็บคำไม่รู้จักลง Sheet 'Unknown_Categories'
def log_unknown_category(message, parsed_category, amount):
    try:
        unknown_sheet = client.open_by_key(SHEET_ID).worksheet("Unknown_Categories")
    except:
        print("❌ ไม่พบ Sheet 'Unknown_Categories'")
        return

    now = datetime.datetime.now().isoformat()
    unknown_sheet.append_row([now, message, parsed_category or "", amount or ""])
    print(f"📥 Logged unknown: '{message}'")

In [11]:
# 🔢 Block 10: sync ข้อมูลแบบเดาหมวด และเก็บ unknown
def sync_sheet_to_notion_dynamic():
    data = sheet.get_all_records()
    for idx, row in enumerate(data):
        if row['Synced?'].strip().upper() != "NO":
            continue

        message = row['Message']
        category, amount = parse_message(message)
        if not category or amount is None:
            print(f"❌ Format ผิด row {idx+2}: '{message}'")
            continue

        guessed_category = find_category(category)
        db_id = category_to_db_id.get(guessed_category)

        if not guessed_category or not db_id:
            print(f"⚠️ หมวดไม่รู้จัก: '{message}'")
            log_unknown_category(message, category, amount)
            continue

        try:
            notion.pages.create(
                parent={"database_id": db_id},
                properties={
                    "Name": {"title": [{"text": {"content": message}}]},
                    "Amount": {"number": amount}
                }
            )
            sheet.update_cell(idx + 2, 4, "YES")
            print(f"✅ Synced row {idx+2}: '{message}' → '{guessed_category}'")
        except Exception as e:
            print(f"🚨 Error row {idx+2}: {e}")

In [12]:
# 🔢 Block 11: log & sync ข้อความเดียว (เช่นพิมพ์จาก LINE)
def log_and_sync_message(user_id, message):
    now = datetime.datetime.now().isoformat()
    sheet.append_row([now, user_id, message, "NO"])

    parsed_text, amount = parse_message(message)
    if not parsed_text or amount is None:
        return f"❌ ข้อความผิด: '{message}'"

    guessed_category = find_category(parsed_text)
    if not guessed_category:
        log_unknown_category(message, parsed_text, amount)
        return f"⚠️ ไม่รู้จักหมวด: '{message}'"

    db_id = category_to_db_id.get(guessed_category)
    if not db_id:
        return f"❌ ไม่มี Database ID สำหรับ '{guessed_category}'"

    try:
        notion.pages.create(
            parent={"database_id": db_id},
            properties={
                "Name": {"title": [{"text": {"content": message}}]},
                "Amount": {"number": amount}
            }
        )
        data = sheet.get_all_records()
        for i, row in enumerate(data):
            if row["Message"] == message and row["UserID"] == user_id:
                sheet.update_cell(i + 2, 4, "YES")
                break

        return f"✅ บันทึกแล้ว: '{message}' → หมวด '{guessed_category}'"
    except Exception as e:
        return f"🚨 เกิดข้อผิดพลาด: {e}"

In [13]:
# 🔧 ติดตั้ง SDK เวอร์ชันเดิม (รองรับ LineBotApi, WebhookHandler)
!pip install line-bot-sdk==1.20.0

In [14]:
# Block 12: LINE Messaging API แบบคลาสสิก (v1.x)
from linebot import LineBotApi, WebhookHandler
from linebot.models import MessageEvent, TextMessage, TextSendMessage
import os

LINE_CHANNEL_ACCESS_TOKEN = os.getenv("LINE_TOKEN")
LINE_CHANNEL_SECRET = os.getenv("LINE_CHANNEL_SECRET")

line_bot_api = LineBotApi(LINE_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)

In [15]:
# 🔢 Block 13: Handle ข้อความจาก LINE + log + ตอบกลับ
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    msg = event.message.text

    # ✅ Log ข้อความจาก user
    log_conversation(user_id, "user", msg)

    # ✅ ตอบกลับด้วยฟังก์ชันรวม
    reply = log_and_sync_message(user_id, msg)

    # ✅ Log ข้อความจาก bot
    log_conversation(user_id, "bot", reply)

    # ✅ ตอบกลับผ่าน LINE
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply)
    )